In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[2]:


import glob

import io_subs
import numpy as np
n_tot=0

import xarray as xr
def write_subset(z_meas_L,p_type_L,bin_nodes_L,sfc_bin_L,p_rate_cmb_L,dm_L,bin_zero_deg_L,bin_storm_top_L,orbit_number,surface_type,pia_kuka_L,pia_srt_L,z_true_L):
    ds=xr.Dataset()
    ds["z_meas"]=xr.DataArray(z_meas_L,dims=["n","n_bins"])
    ds["p_type"]=xr.DataArray(p_type_L,dims=["n"])
    ds["bin_nodes"]=xr.DataArray(bin_nodes_L,dims=["n","n5"])
    ds["sfc_bin"]=xr.DataArray(sfc_bin_L,dims=["n","n2"])
    ds["p_rate_cmb"]=xr.DataArray(p_rate_cmb_L,dims=["n","n_bins_cmb"])
    ds["dm"]=xr.DataArray(dm_L,dims=["n","n_bins_cmb"])
    ds["z_true"]=xr.DataArray(z_true_L,dims=["n","n_bins_cmb"])
    ds["pia"]=xr.DataArray(pia_kuka_L,dims=["n","n_chan"])
    ds["pia_srt"]=xr.DataArray(pia_srt_L,dims=["n","n_chan"])
    ds["bin_zero_deg"]=xr.DataArray(bin_zero_deg_L,dims=["n"])
    ds["bin_storm_top"]=xr.DataArray(bin_storm_top_L,dims=["n"])
    ds["surface_type"]=xr.DataArray(surface_type,dims=["n"])
    comp = dict(zlib=True, complevel=5)
    ds.to_netcdf("output/subset_%s.nc"%orbit_number,encoding={"z_meas":comp,"p_type":comp,"bin_nodes":comp,"sfc_bin":comp,"p_rate_cmb":comp,"dm":comp,"bin_zero_deg":comp,"bin_storm_top":comp,"surface_type":comp,"pia":comp,"pia_srt":comp,"z_true":comp})
    return

import netCDF4 as nc
import pyresample
for im in [1]:
    f1=glob.glob("../../../sringeru/ITE_790/2020/%2.2i/*"%im)
    print(len(f1))
    f2=glob.glob("../../../sringeru/1C_GMI/2020/%2.2i/*"%im)
    print(len(f2))
    


    f1=sorted(f1)
    f2=sorted(f2)

    dict_cmb={}
    for fcmb in f1[::3]:
        orb1=fcmb.split(".")[-3]
        ifound=0
        for f2t in f2:
            if orb1==f2t.split(".")[-3]:
                f_gmi=f2t
                ifound=1
                break
        if ifound==1:
            print(fcmb,f_gmi)
        else:
            continue
        z_meas_L=[]
        p_type_L=[]
        bin_nodes_L=[]
        sfc_bin_L=[]
        p_rate_cmb_L=[]
        dm_L=[]
        bin_zero_deg_L=[]
        bin_storm_top_L=[]
        surface_type_L=[]
        pia_kuka_L=[]
        pia_srt_L=[]
        z_true_L=[]
        qv,press,envNodes,airTemp,skTemp,binNodes,sfcEmiss,sfcBin,pType,lon,lat,oe_wvp,oe_lwp,oe_iwp,oesfc_precip,stormTop,nearSfcPrecip,zeroDegAltitude,surfaceType=io_subs.readCMB_2(fcmb)
        pType=np.array(pType/1e7,dtype=int)
        a=np.nonzero(pType>0)
        n_tot+=len(a[0])

        lat_s1,lon_s1,tc_s1,lat_s2,lon_s2,tc_s2=io_subs.read1CGMI(f_gmi)
        nscans=qv.shape[0]
        nt=nscans//128
        tc_s1_L=[]
        tc_s2_L=[]
        lon_L=[]
        lat_L=[]
        qv_L=[]
        press_L=[]
        airTemp_L=[]
        sfc_bin_L=[]
        sfc_emiss_L=[]
        oe_wvp_L=[]
        oe_lwp_L=[]
        oe_iwp_L=[]
        surface_type_L=[]
        near_sfc_precip_L=[]
        sk_temp_L=[]
        for ich in range(nt):
            try:
                grid_def = pyresample.geometry.GridDefinition(lons=lon.data[ich*128:(ich+1)*128,:], lats=lat.data[ich*128:(ich+1)*128,:])
                rad_inf=10000.0
                swath_def = pyresample.geometry.SwathDefinition(lons=lon_s1.data, lats=lat_s1.data)
                tc_s1_grid = pyresample.kd_tree.resample_gauss(swath_def, tc_s1[:,:,:], grid_def, radius_of_influence=rad_inf, fill_value=-999, sigmas=[6000 for k in range(9)])
                swath_def = pyresample.geometry.SwathDefinition(lons=lon_s2.data, lats=lat_s2.data)
                tc_s2_grid = pyresample.kd_tree.resample_gauss(swath_def, tc_s2[:,:,:], grid_def, radius_of_influence=rad_inf, fill_value=-999, sigmas=[6000 for k in range(4)])
                tc_s1_L.append(tc_s1_grid)
                tc_s2_L.append(tc_s2_grid)
                lon_L.append(lon.data[ich*128:(ich+1)*128,:])
                lat_L.append(lat.data[ich*128:(ich+1)*128,:])
                qv_L.append(qv.data[ich*128:(ich+1)*128,:])
                press_L.append(press.data[ich*128:(ich+1)*128,:])
                airTemp_L.append(airTemp.data[ich*128:(ich+1)*128,:])
                sfc_bin_L.append(sfcBin.data[ich*128:(ich+1)*128,:])
                sfc_emiss_L.append(sfcEmiss.data[ich*128:(ich+1)*128,:])
                oe_wvp_L.append(oe_wvp.data[ich*128:(ich+1)*128,:])
                oe_lwp_L.append(oe_lwp.data[ich*128:(ich+1)*128,:])
                oe_iwp_L.append(oe_iwp.data[ich*128:(ich+1)*128,:])
                near_sfc_precip_L.append(nearSfcPrecip.data[ich*128:(ich+1)*128,:])
                sk_temp_L.append(skTemp.data[ich*128:(ich+1)*128,:])
                surface_type_L.append(surfaceType.data[ich*128:(ich+1)*128,:])
            except:
                continue
            if ich==5:    
                break
        save_patches(tc_s1_L,tc_s2_L,lon_L,lat_L,qv_L,press_L,airTemp_L,sfc_bin_L,sfc_emiss_L,oe_wvp_L,oe_lwp_L,oe_iwp_L,near_sfc_precip_L,orb1)
        break

import xarray as xr
def save_patches(tc_s1_L,tc_s2_L,lon_L,lat_L,qv_L,press_L,airTemp_L,sfc_bin_L,sfc_emiss_L,oe_wvp_L,oe_lwp_L,oe_iwp_L,near_sfc_precip_L,sk_temp_L,surface_type_L,orbit_number):
    ds=xr.Dataset()
    ds["tc_s1"]=xr.DataArray(np.array(tc_s1_L,dtype=np.float16), dims=["n_chunks","nx","ny","n_s1"])
    ds["tc_s2"]=xr.DataArray(np.array(tc_s2_L,dtype=np.float16),dims=["n_chunks","nx","ny","n_s2"])
    ds["lon"]=xr.DataArray(lon_L,dims=["n_chunks","nx","ny"])
    ds["lat"]=xr.DataArray(lat_L,dims=["n_chunks","nx","ny"])
    ds["qv"]=xr.DataArray(np.array(qv_L),dims=["n_chunks","nx","ny","nenv"])
    ds["press"]=xr.DataArray(np.array(press_L,dtype=np.float16),dims=["n_chunks","nx","ny","nenv"])
    ds["airTemp"]=xr.DataArray(np.array(airTemp_L,dtype=np.float16),dims=["n_chunks","nx","ny","nenv"])
    ds["sfc_bin"]=xr.DataArray(np.array(sfc_bin_L,dtype=np.int16)),dims=["n_chunks","nx","ny"])
    ds["sfc_emiss"]=xr.DataArray(np.array(sfc_emiss_L,dtype=np.float16),dims=["n_chunks","nx","ny","n_s1_s2"])
    ds["oe_wvp"]=xr.DataArray(np.array(oe_wvp_L,dtype=np.float16),dims=["n_chunks","nx","ny"])
    ds["oe_lwp"]=xr.DataArray(np.array(oe_lwp_L,dtype=np.float16),dims=["n_chunks","nx","ny"])
    ds["oe_iwp"]=xr.DataArray(np.array(oe_iwp_L,dtype=np.float16),dims=["n_chunks","nx","ny"])
    ds["near_sfc_precip"]=xr.DataArray(np.array(near_sfc_precip_L,,dtype=np.float16),dims=["n_chunks","nx","ny"])
    ds["sk_temp"]=xr.DataArray(np.array(sk_temp_L,dtype=np.float16),dims=["n_chunks","nx","ny","nenv"])
    ds["surface_type"]=xr.DataArray(np.array(surface_type_L,dtype=np.int16),dims=["n_chunks","nx","ny"])
    comp = dict(zlib=True, complevel=5)
    ds.to_netcdf("output/patches_%s.nc"%orbit_number,encoding={"tc_s1":comp,"tc_s2":comp,"lon":comp,"lat":comp,"qv":comp,"press":comp,"airTemp":comp,"sfc_bin":comp,"sfc_emiss":comp,"oe_wvp":comp,"oe_lwp":comp,"oe_iwp":comp,"near_sfc_precip":comp,"sk_temp":comp,"surface_type":comp})
    return